<a href="https://colab.research.google.com/github/rajibmitra/gemini-workshop/blob/main/func-calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [3]:
from google.colab import auth
auth.authenticate_user()

project_id = 'gemini-workshop-420907'
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].


In [5]:
!pip install --upgrade google-cloud-aiplatform


In [10]:
import vertexai
vertexai.init(project=project_id,location="us-central1")
from vertexai.generative_models import GenerativeModel
model = GenerativeModel("gemini-1.0-pro")

In [11]:
response = model.generate_content(
    "What's the exchange rate for euros to dollars today?"
)
print(response.text)

The current exchange rate for euros to dollars as of Friday, October 27, 2023, is 1 euro equals 1.17 US dollars. This means that you would need 1.17 US dollars to buy one euro. 

It's important to note that this is just the current exchange rate and it can fluctuate throughout the day. The exchange rate is also affected by several factors, including economic conditions, political events, and market sentiment.

If you are planning to exchange euros for dollars, I recommend checking with a currency exchange service or your bank to get the most up-to-date exchange rate. You should also factor in any fees or commissions that may be charged.


In [12]:
user_prompt = "What's the exchange rate from euros to US dollars today?"

response = model.generate_content("""
Your task is to extract parameters from the user's input and return it as a
structured JSON payload. The user will ask about the exchange rate and which
currency they are converting from and converting to.

User input: {user_prompt}

Please extract the currencies as parameters and put them in a JSON object.
""".format(user_prompt=user_prompt))
print(response.text)

```json
{
  "fromCurrency": "euros",
  "toCurrency": "US dollars"
}
```


In [13]:
import requests
url = "https://api.frankfurter.app/latest"
response = requests.get(url)
response.text

'{"amount":1.0,"base":"EUR","date":"2024-04-19","rates":{"AUD":1.6622,"BGN":1.9558,"BRL":5.6165,"CAD":1.4666,"CHF":0.968,"CNY":7.7139,"CZK":25.267,"DKK":7.4622,"GBP":0.8562,"HKD":8.3443,"HUF":395.3,"IDR":17325,"ILS":4.0181,"INR":88.92,"ISK":150.3,"JPY":164.68,"KRW":1469.09,"MXN":18.4819,"MYR":5.0959,"NOK":11.765,"NZD":1.8105,"PHP":61.373,"PLN":4.33,"RON":4.9764,"SEK":11.664,"SGD":1.451,"THB":39.288,"TRY":34.73,"USD":1.0653,"ZAR":20.479}}'

In [15]:
!curl https://api.frankfurter.app/latest


{"amount":1.0,"base":"EUR","date":"2024-04-19","rates":{"AUD":1.6622,"BGN":1.9558,"BRL":5.6165,"CAD":1.4666,"CHF":0.968,"CNY":7.7139,"CZK":25.267,"DKK":7.4622,"GBP":0.8562,"HKD":8.3443,"HUF":395.3,"IDR":17325,"ILS":4.0181,"INR":88.92,"ISK":150.3,"JPY":164.68,"KRW":1469.09,"MXN":18.4819,"MYR":5.0959,"NOK":11.765,"NZD":1.8105,"PHP":61.373,"PLN":4.33,"RON":4.9764,"SEK":11.664,"SGD":1.451,"THB":39.288,"TRY":34.73,"USD":1.0653,"ZAR":20.479}}

In [16]:
from vertexai.generative_models import (
    Content,
    FunctionDeclaration,
    GenerativeModel,
    Part,
    Tool,
)

model = GenerativeModel("gemini-1.0-pro")

In [17]:
get_exchange_rate_func = FunctionDeclaration(
    name="get_exchange_rate",
    description="Get the exchange rate for currencies between countries",
    parameters={
    "type": "object",
    "properties": {
        "currency_date": {
            "type": "string",
            "description": "A date that must always be in YYYY-MM-DD format or the value 'latest' if a time period is not specified"
        },
        "currency_from": {
            "type": "string",
            "description": "The currency to convert from in ISO 4217 format"
        },
        "currency_to": {
            "type": "string",
            "description": "The currency to convert to in ISO 4217 format"
        }
    },
         "required": [
            "currency_from",
            "currency_date",
      ]
  },
)

In [18]:
exchange_rate_tool = Tool(
    function_declarations=[get_exchange_rate_func],
)

In [19]:
prompt = """What is the exchange rate from Australian dollars to Swedish krona?
How much is 500 Australian dollars worth in Swedish krona?"""

response = model.generate_content(
    prompt,
    tools=[exchange_rate_tool],
)

In [20]:
print(response.candidates[0].content)

role: "model"
parts {
  function_call {
    name: "get_exchange_rate"
    args {
      fields {
        key: "currency_date"
        value {
          string_value: "latest"
        }
      }
      fields {
        key: "currency_from"
        value {
          string_value: "AUD"
        }
      }
      fields {
        key: "currency_to"
        value {
          string_value: "SEK"
        }
      }
    }
  }
}



In [21]:
params = {}
for key, value in response.candidates[0].content.parts[0].function_call.args.items():
    params[key[9:]] = value
params

{'date': 'latest', 'to': 'SEK', 'from': 'AUD'}

In [22]:
import requests
url = f"https://api.frankfurter.app/{params['date']}"
api_response = requests.get(url, params=params)
api_response.text

'{"amount":1.0,"base":"AUD","date":"2024-04-19","rates":{"SEK":7.0172}}'

In [23]:
response = model.generate_content(
    [
    Content(role="user", parts=[
        Part.from_text(prompt + """Give your answer in steps with lots of detail
            and context, including the exchange rate and date."""),
    ]),
    Content(role="function", parts=[
        Part.from_dict({
            "function_call": {
                "name": "get_exchange_rate",
            }
        })
    ]),
    Content(role="function", parts=[
        Part.from_function_response(
            name="get_exchange_rate",
            response={
                "content": api_response.text,
            }
        )
    ]),
    ],
    tools=[exchange_rate_tool],
)


response.candidates[0].content.parts[0].text

'## Exchange Rate from AUD to SEK on 2024-04-19\n\nThe current exchange rate from Australian dollars (AUD) to Swedish krona (SEK) is 1 AUD = 7.0172 SEK. This means that 1 Australian dollar can be exchanged for 7.0172 Swedish kronor.\n\n## How much is 500 AUD worth in SEK?\n\n500 AUD is worth 7.0172 * 500 = 3508.6 SEK. Therefore, 500 Australian dollars can be exchanged for 3508.6 Swedish kronor on April 19th, 2024. \n\n**Note:** Exchange rates can fluctuate throughout the day, so this is just an estimate based on the current rate.\n'